In [54]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [55]:
import pandas as pd
from geopy.geocoders import nominatim

In [56]:
df = pd.read_csv("data.csv",sep=',')

In [57]:
df.head()

,GLOBAL_SITE_ID,SITE_NAME,SITE_TYPE,STREET_ADDRESS,CITY,STATE,POSTAL_CODE,COUNTRY
0,10000253,1241 BOYLSTON ST,Shell In Network Retail Site,1241 BOYLSTON ST,BOSTON,Massachusetts,02215-3501,USA
1,10000254,1975 MAIN ST,Shell In Network Retail Site,1975 MAIN ST,TEWKSBURY,Massachusetts,01876-2176,USA
2,10000256,190 AMHERST ST,Shell In Network Retail Site,190 AMHERST ST,NASHUA,New Hampshire,03064-1307,USA
3,10000257,915 WALTHAM ST,Shell In Network Retail Site,915 WALTHAM ST,LEXINGTON,Massachusetts,02421-8001,USA
4,10000258,245 EDDY RD,Shell In Network Retail Site,245 EDDY RD,MANCHESTER,New Hampshire,03102-3232,USA


In [58]:
#df['query'] = df['COMPANY'] + " " + df['CITY'] + " " +  df['COUNTRY']
df['query'] = df['STREET_ADDRESS'] + " " + df['CITY'] + " " + df['STATE'] + " " + df['POSTAL_CODE'] + df['COUNTRY']
df.iloc[0:5, 8:9]   

,query
0,1241 BOYLSTON ST BOSTON Massachusetts 02215-35...
1,1975 MAIN ST TEWKSBURY Massachusetts 01876-217...
2,190 AMHERST ST NASHUA New Hampshire 03064-1307USA
3,915 WALTHAM ST LEXINGTON Massachusetts 02421-8...
4,245 EDDY RD MANCHESTER New Hampshire 03102-323...


In [59]:
#remove duplicate addresses (new concat column) but keep first instance
df.drop_duplicates(subset ='query', keep ='first', inplace = True)

#print some useful info: row length and shape size
print("data row x columns is {}\ndata row count is {}".format(df.shape,len(df.index)))

#print first rows as sample
df.head()

data row x columns is (12, 9)
data row count is 12


,GLOBAL_SITE_ID,SITE_NAME,SITE_TYPE,STREET_ADDRESS,CITY,STATE,POSTAL_CODE,COUNTRY,query
0,10000253,1241 BOYLSTON ST,Shell In Network Retail Site,1241 BOYLSTON ST,BOSTON,Massachusetts,02215-3501,USA,1241 BOYLSTON ST BOSTON Massachusetts 02215-35...
1,10000254,1975 MAIN ST,Shell In Network Retail Site,1975 MAIN ST,TEWKSBURY,Massachusetts,01876-2176,USA,1975 MAIN ST TEWKSBURY Massachusetts 01876-217...
2,10000256,190 AMHERST ST,Shell In Network Retail Site,190 AMHERST ST,NASHUA,New Hampshire,03064-1307,USA,190 AMHERST ST NASHUA New Hampshire 03064-1307USA
3,10000257,915 WALTHAM ST,Shell In Network Retail Site,915 WALTHAM ST,LEXINGTON,Massachusetts,02421-8001,USA,915 WALTHAM ST LEXINGTON Massachusetts 02421-8...
4,10000258,245 EDDY RD,Shell In Network Retail Site,245 EDDY RD,MANCHESTER,New Hampshire,03102-3232,USA,245 EDDY RD MANCHESTER New Hampshire 03102-323...


In [60]:
#drop columns we won't use
df = df.drop(columns=['GLOBAL_SITE_ID','SITE_NAME','SITE_TYPE'])
                 
#print row length and shape size
print("data row x columns is {}\ndata row count is {}".format(df.shape,len(df.index)))

#print first rows as sample
df.head()

data row x columns is (12, 6)
data row count is 12


,STREET_ADDRESS,CITY,STATE,POSTAL_CODE,COUNTRY,query
0,1241 BOYLSTON ST,BOSTON,Massachusetts,02215-3501,USA,1241 BOYLSTON ST BOSTON Massachusetts 02215-35...
1,1975 MAIN ST,TEWKSBURY,Massachusetts,01876-2176,USA,1975 MAIN ST TEWKSBURY Massachusetts 01876-217...
2,190 AMHERST ST,NASHUA,New Hampshire,03064-1307,USA,190 AMHERST ST NASHUA New Hampshire 03064-1307USA
3,915 WALTHAM ST,LEXINGTON,Massachusetts,02421-8001,USA,915 WALTHAM ST LEXINGTON Massachusetts 02421-8...
4,245 EDDY RD,MANCHESTER,New Hampshire,03102-3232,USA,245 EDDY RD MANCHESTER New Hampshire 03102-323...


In [61]:
#create 2 new columns to store lat/long - initalise to null
df['location_lat'] = ""
df['location_long'] = ""
df['location_address'] = ""

#print first rows to sample
df.head()

,STREET_ADDRESS,CITY,STATE,POSTAL_CODE,COUNTRY,query,location_lat,location_long,location_address
0,1241 BOYLSTON ST,BOSTON,Massachusetts,02215-3501,USA,1241 BOYLSTON ST BOSTON Massachusetts 02215-35...,,,
1,1975 MAIN ST,TEWKSBURY,Massachusetts,01876-2176,USA,1975 MAIN ST TEWKSBURY Massachusetts 01876-217...,,,
2,190 AMHERST ST,NASHUA,New Hampshire,03064-1307,USA,190 AMHERST ST NASHUA New Hampshire 03064-1307USA,,,
3,915 WALTHAM ST,LEXINGTON,Massachusetts,02421-8001,USA,915 WALTHAM ST LEXINGTON Massachusetts 02421-8...,,,
4,245 EDDY RD,MANCHESTER,New Hampshire,03102-3232,USA,245 EDDY RD MANCHESTER New Hampshire 03102-323...,,,


In [63]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="myApp")

for i in df.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(df['query'][i])
        
        #append lat/long to column using dataframe location
        df.loc[i,'location_lat'] = location.latitude
        df.loc[i,'location_long'] = location.longitude
        df.loc[i,'location_address'] = location.address
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        df.loc[i,'location_lat'] = ""
        df.loc[i,'location_long'] = ""
        df.loc[i,'location_address'] = ""

#print first rows as sample
df.head()

,STREET_ADDRESS,CITY,STATE,POSTAL_CODE,COUNTRY,query,location_lat,location_long,location_address
0,1241 BOYLSTON ST,BOSTON,Massachusetts,02215-3501,USA,1241 BOYLSTON ST BOSTON Massachusetts 02215-35...,,,
1,1975 MAIN ST,TEWKSBURY,Massachusetts,01876-2176,USA,1975 MAIN ST TEWKSBURY Massachusetts 01876-217...,,,
2,190 AMHERST ST,NASHUA,New Hampshire,03064-1307,USA,190 AMHERST ST NASHUA New Hampshire 03064-1307USA,,,
3,915 WALTHAM ST,LEXINGTON,Massachusetts,02421-8001,USA,915 WALTHAM ST LEXINGTON Massachusetts 02421-8...,,,
4,245 EDDY RD,MANCHESTER,New Hampshire,03102-3232,USA,245 EDDY RD MANCHESTER New Hampshire 03102-323...,,,


In [ ]:
#write the contents thus far to new csv file
df.to_csv('geopy_data.csv')